In [3]:
import numpy as np

In [4]:
class Venus:
    
    def __init__(
        self,
        inj_limits=[175, 185],
        mid_limits=[145, 155],
        ext_limits=[135, 145],
        beam_range=[50, 100],
        jitter=0
    ):
        """The limits on the magnetic solenoids currents and the beam range (ouput).
        A random jitter can be added also (fraction of 1.)."""
        self.inj_limits = inj_limits
        self.mid_limits = mid_limits
        self.ext_limits = ext_limits
        self.beam_range = beam_range
        self.currents = np.zeros(3)
        self.jitter = jitter
        self.rng = np.random.default_rng(42)

    def set_mag_currents(self, inj, mid, ext):
        """Set the magnetic currents on the coils."""
        for v, lim in zip([inj, mid, ext], [self.inj_limits, self.mid_limits, self.ext_limits]):
            if v < lim[0] or v > lim[1]:
                raise ValueError("Setting outside limits")
        self.currents = np.array([inj, mid, ext])

    def _rescale_inputs(self, inputs):
        """input to himmelblau4 must be in [-6, 6]."""
        return (
            (c - l[0]) * 12.0 / (l[1] - l[0]) - 6.0
            for c, l in zip(inputs, [self.inj_limits, self.mid_limits, self.ext_limits])
        )

    def _rescale_output(self, output):
        """himmelblau4 returns values betwen 0 and 4899 for w, x, y, z in [-6, 6]."""
        return (
            (1. - (output / 4899) + self.rng.normal(0.0, self.jitter)) *
            (self.beam_range[1] - self.beam_range[0]) + self.beam_range[0]
        )

    def get_beam_current(self):
        """Read the current value of the beam current"""
        return self._rescale_output(self._himmelblau4(*self._rescale_inputs(self.currents)))

    @staticmethod
    def _himmelblau4(w, x, y):
        """A funky 4 dimensional parameter space with a bunch of local minima."""
        return (
            (w**2 + x + y - 11)**2 +
            (w + x**2 + y - 7)**2 +
            (w + x + y**2 - 5)**2
        )

create a venus object, defaults might be refined by values from Damon. For simpler algorithm the jitter can be set to 0 initially but it should work with a value larger than that in the end.

In [5]:
venus = Venus(jitter=0.05)

set the currents and read the beam value

In [6]:
venus.set_mag_currents(185, 155, 145)
bc = venus.get_beam_current()
print(bc)

50.76179269938608


if currents are outside the range a `ValueError` is raised and can be captured.

In [7]:
venus.set_mag_currents(50, 200, 200)

ValueError: Setting outside limits

In [8]:
venus._himmelblau4(6, 6, 6)

4899

# Bayesian Optimization

In [10]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from bayes_opt import BayesianOptimization, UtilityFunction
import warnings
warnings.filterwarnings("ignore")

# Define the black box function to optimize.
def black_box_function(A, B, C):
    # C: SVC hyper parameter to optimize for.
    v = venus._himmelblau4(A, B, C)
    return v

# Set range of C to optimize for.
# bayes_opt requires this to be a dictionary.
pbounds = {"A": [0, 185], "B": [0, 155], "C": [0, 145]}
# Create a BayesianOptimization optimizer,
# and optimize the given black_box_function.
optimizer = BayesianOptimization(f = black_box_function,
                                 pbounds = pbounds, verbose = 2,
                                 random_state = 4)
optimizer.maximize(init_points = 5, n_iter = 10)
print("Best result: {}; f(x) = {}.".format(optimizer.max["params"], optimizer.max["target"]))

|   iter    |  target   |     A     |     B     |     C     |
-------------------------------------------------------------
|  1        |  1.501e+0 |  178.9    |  84.82    |  141.0    |
|  2        |  4.523e+0 |  132.2    |  108.1    |  31.33    |
|  3        |  1.068e+0 |  180.6    |  0.9657   |  36.68    |
|  4        |  2.607e+0 |  80.44    |  120.8    |  28.66    |
|  5        |  1.207e+0 |  159.7    |  152.4    |  23.76    |
|  6        |  1.413e+0 |  175.6    |  82.42    |  140.5    |
|  7        |  1.882e+0 |  185.0    |  122.9    |  145.0    |
|  8        |  1.756e+0 |  184.9    |  149.4    |  87.09    |
|  9        |  4.419e+0 |  0.0      |  0.0      |  145.0    |
|  10       |  2.24e+09 |  185.0    |  155.0    |  145.0    |
|  11       |  1.032e+0 |  0.0      |  155.0    |  145.0    |
|  12       |  1.476e+0 |  143.0    |  155.0    |  145.0    |
|  13       |  1.63e+09 |  185.0    |  0.0      |  145.0    |
|  14       |  1.97e+09 |  183.1    |  151.1    |  129.5    |
|  15   